In [1]:
from astroquery.gaia import Gaia

In [30]:
GaiaCatalog = "gaiadr3.gaia_source" 
PanSTARRSMatchCatalog = "gaiadr3.panstarrs1_best_neighbour"
PanSTARRSCatalog = "gaiadr2.panstarrs1_original_valid"
TMASSMatchCatalog = "gaiadr3.tmass_psc_xsc_best_neighbour"
TMASSJoinCatalog = "gaiadr3.tmass_psc_xsc_join"
TMASSCatalog = "gaiadr1.tmass_original_valid"

maxPMerror = 5 # mas/year

In [69]:
columns = ['gaia.source_id',
    'gaia.ra',
    'gaia.dec',
    'gaia.pmra',
    'gaia.pmdec',
    'gaia.radial_velocity',
    'gaia.phot_g_mean_mag',
    'gaia.phot_g_mean_flux_over_error',
    'gaia.phot_bp_mean_mag',
    'gaia.phot_bp_mean_flux_over_error',
    'gaia.phot_rp_mean_mag',
    'gaia.phot_rp_mean_flux_over_error',
    'gaia.parallax',
    'gaia.teff_gspphot',
    'gaia.ruwe',
    'best.number_of_neighbours',
    'best.number_of_mates',
    'ps.g_mean_psf_mag',
    'ps.g_mean_psf_mag_error',
    'ps.r_mean_psf_mag',
    'ps.r_mean_psf_mag_error',
    'ps.i_mean_psf_mag',
    'ps.i_mean_psf_mag_error',
    'ps.z_mean_psf_mag',
    'ps.z_mean_psf_mag_error',
    'ps.y_mean_psf_mag',
    'ps.y_mean_psf_mag_error',
    'tmass.j_m',
    'tmass.j_msigcom',
    'tmass.h_m',
    'tmass.h_msigcom',
    'tmass.ks_m',
    'tmass.ks_msigcom',
]

columns = ', '.join(columns)

In [70]:
RA = 12.108
Dec = 85.255
radius = 0.1

In [72]:
ADQLcmd = f"SELECT {columns} " + \
    f"FROM {GaiaCatalog} AS gaia " + \
    f"LEFT OUTER JOIN {PanSTARRSMatchCatalog} AS best ON gaia.source_id = best.source_id " +  \
    f"LEFT OUTER JOIN {PanSTARRSCatalog} AS ps ON best.original_ext_source_id = ps.obj_id " +  \
    f"LEFT OUTER JOIN {TMASSMatchCatalog} AS xmatch ON gaia.source_id = xmatch.source_id " + \
    f"LEFT OUTER JOIN {TMASSJoinCatalog} AS xjoin ON xmatch.clean_tmass_psc_xsc_oid = xjoin.clean_tmass_psc_xsc_oid " + \
    f"LEFT OUTER JOIN {TMASSCatalog} AS tmass ON xjoin.original_psc_source_id = tmass.designation " + \
    f"WHERE CONTAINS(POINT('ICRS',gaia.ra, gaia.dec)," + \
    f"CIRCLE('ICRS', {RA}, {Dec}, {radius}))=1 " + \
    f"AND abs(gaia.pmra_error)<{maxPMerror} " + \
    f"AND abs(gaia.pmdec_error)<{maxPMerror} " + \
    f"AND gaia.pmra IS NOT NULL AND abs(gaia.pmra)>0 " + \
    f"AND gaia.pmdec IS NOT NULL AND abs(gaia.pmdec)>0;"

print(ADQLcmd)

SELECT gaia.source_id, gaia.ra, gaia.dec, gaia.pmra, gaia.pmdec, gaia.radial_velocity, gaia.phot_g_mean_mag, gaia.phot_g_mean_flux_over_error, gaia.phot_bp_mean_mag, gaia.phot_bp_mean_flux_over_error, gaia.phot_rp_mean_mag, gaia.phot_rp_mean_flux_over_error, gaia.parallax, gaia.teff_gspphot, gaia.ruwe, best.number_of_neighbours, best.number_of_mates, ps.g_mean_psf_mag, ps.g_mean_psf_mag_error, ps.r_mean_psf_mag, ps.r_mean_psf_mag_error, ps.i_mean_psf_mag, ps.i_mean_psf_mag_error, ps.z_mean_psf_mag, ps.z_mean_psf_mag_error, ps.y_mean_psf_mag, ps.y_mean_psf_mag_error, tmass.j_m, tmass.j_msigcom, tmass.h_m, tmass.h_msigcom, tmass.ks_m, tmass.ks_msigcom FROM gaiadr3.gaia_source AS gaia LEFT OUTER JOIN gaiadr3.panstarrs1_best_neighbour AS best ON gaia.source_id = best.source_id LEFT OUTER JOIN gaiadr2.panstarrs1_original_valid AS ps ON best.original_ext_source_id = ps.obj_id LEFT OUTER JOIN gaiadr3.tmass_psc_xsc_best_neighbour AS xmatch ON gaia.source_id = xmatch.source_id LEFT OUTER JOIN g

In [73]:
job = Gaia.launch_job_async(ADQLcmd, dump_to_file=False) #could save this to a file

INFO: Query finished. [astroquery.utils.tap.core]


In [74]:
job.get_results()

source_id,ra,dec,pmra,pmdec,radial_velocity,phot_g_mean_mag,phot_g_mean_flux_over_error,phot_bp_mean_mag,phot_bp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_mean_flux_over_error,parallax,teff_gspphot,ruwe,number_of_neighbours,number_of_mates,g_mean_psf_mag,g_mean_psf_mag_error,r_mean_psf_mag,r_mean_psf_mag_error,i_mean_psf_mag,i_mean_psf_mag_error,z_mean_psf_mag,z_mean_psf_mag_error,y_mean_psf_mag,y_mean_psf_mag_error,j_m,j_msigcom,h_m,h_msigcom,ks_m,ks_msigcom
,deg,deg,mas / yr,mas / yr,km / s,mag,,mag,,mag,,mas,K,,,,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag
int64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,int16,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32
573934800434741760,11.800092936880631,85.16020643413596,-0.34260109726566945,0.3201267706047321,--,17.172804,1408.4047,17.553349,134.90123,16.635153,198.32681,0.3261537220765713,5300.8286,0.9965295,1,0,17.6117000579834,0.00466499989852309,17.1625995635986,0.00486399978399277,16.9913997650146,0.00327500002458692,16.9302005767822,0.00621599983423948,16.9109001159668,0.0107330000028014,16.028,0.085,15.661,0.156,15.815,--
573934864858240768,11.609202804602221,85.16416856123969,-2.3158629917809055,4.973691214809051,--,17.990503,945.2018,18.364067,51.761173,17.490757,84.543755,0.40025107738482374,5394.4175,1.017709,1,0,18.3770999908447,0.00664300005882978,17.9957008361816,0.00420300010591745,17.8090000152588,0.00698499986901879,17.7558994293213,0.00748099992051721,17.7511005401611,0.0174920000135899,--,--,--,--,--,--
573934869154213632,11.639762625418992,85.16800977224642,-7.6985324404469235,5.71910670045043,--,14.984311,4269.0513,15.390951,496.50836,14.414208,880.42206,1.213878109383751,5434.396,0.9895143,1,0,15.4687004089355,0.00112499995157123,14.9682998657227,0.00314299995079637,14.7840995788574,0.00250299996696413,14.7157001495361,0.00332199991680682,14.671799659729,0.00449299998581409,13.719,0.029,13.291,0.041,13.193,0.035
573935075312637696,11.355546052130615,85.17723972322962,-2.6004675196061586,-1.1891485524897507,-44.54503,14.686383,4884.2104,15.204281,434.0891,14.019403,970.70026,0.5466069585562476,4851.3867,1.2164924,1,0,15.3346996307373,0.00267400010488927,14.6496000289917,0.00205900007858872,14.394100189209,0.00320699997246265,14.2806997299194,0.00449799979105592,14.2093000411987,0.00164200004655868,13.136,0.027,12.64,0.033,12.589,0.021
573935139736156672,11.2434370903608,85.1908007614722,-1.8581652559334978,-1.0205556961383249,--,18.432598,421.48285,19.411655,20.332754,17.456215,64.8789,0.37432672304363956,3557.0269,1.1885147,1,0,19.7171001434326,0.0161230005323887,18.5272998809814,0.00867899972945452,17.8351001739502,0.00642400002107024,17.514799118042,0.00486199976876378,17.3356990814209,0.0120430001989007,16.17,0.124,15.238,0.128,15.481,0.191
573935139736258688,11.240557005443423,85.18937540637748,2.4095500535073637,-1.4376224911404196,--,20.17443,173.2524,20.727804,4.578481,19.06158,19.733812,-0.3708416335886471,--,1.1598091,1,0,--,--,20.7141990661621,0.0608359985053539,19.5118999481201,0.0204499997198582,18.9164009094238,0.0174870006740093,18.6567993164063,0.0208229999989271,--,--,--,--,--,--
573935144030534400,11.19059753870281,85.19348421614625,2.8989958087405774,1.3319666778362032,--,19.946768,210.63202,19.751171,5.5944934,19.377947,8.450742,-0.03629074734460319,--,0.92809325,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
573935144032103936,11.179759791770167,85.1936668356247,-2.25749592966012,-1.021630468067295,--,15.15463,3634.3066,15.5058565,522.0692,14.637541,713.848,0.5169725547905046,--,0.96404964,1,0,15.5595998764038,0.0031910000834614,15.1409997940063,0.00141000002622604,14.9990997314453,0.00144400005228817,14.9561996459961,0.00309600005857646,14.9366998672485,0.00459200004115701,14.016,0.033,13.776,0.044,13.646,0.039
